In [1]:
import qmcpy as qp
import numpy as np
import math
import time

print('okay')

okay


In [2]:
n=10
d=2
i = np.arange(1, n+1).reshape((n, 1))
alpha = np.random.rand(d)
delta = np.random.rand(d)
print(alpha)
print(delta)
print(((i*alpha) + delta)%1)

[0.41450828 0.36400742]
[0.78589229 0.12412424]
[[0.20040057 0.48813166]
 [0.61490885 0.85213907]
 [0.02941712 0.21614649]
 [0.4439254  0.58015391]
 [0.85843368 0.94416132]
 [0.27294196 0.30816874]
 [0.68745023 0.67217616]
 [0.10195851 0.03618357]
 [0.51646679 0.40019099]
 [0.93097506 0.76419841]]


In [3]:
print(qp.kronecker(10, 2))

AttributeError: module 'qmcpy' has no attribute 'kronecker'